### Initialize ellipsoids with DPD

In [2]:
import gsd, gsd.hoomd 
import hoomd
import mbuild as mb
import numpy as np
import scipy.stats
import unyt as u
import warnings
from flowermd.base import Pack,Lattice, Simulation,BaseHOOMDForcefield,Polymer
from flowermd.base.system import System
#from flowermd.library import EllipsoidForcefield, EllipsoidChain
#from flowermd.utils import get_target_box_number_density
from flowermd.utils.constraints import create_rigid_ellipsoid_chain
from scipy.spatial.distance import pdist
warnings.filterwarnings('ignore')

/Users/stephaniemccallum/miniforge3/envs/mupt-examples/lib/python3.12/site-packages/gmso/core/element.py:10: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this package or pin to Setuptools<81.
  from pkg_resources import resource_filename


In [18]:
class EllipsoidChainRand(Polymer):
    def __init__(
        self,
        lengths,
        num_mols,
        lpar,
        bead_mass,
        density,
        bond_L=0.1, #T-T bond length
        name="ellipsoid_chain",
    ):
        self.bead_mass = bead_mass
        self.lpar = lpar
        self.bond_L = bond_L
        self.density = density
        N=lengths*num_mols
        L = np.cbrt(N/ self.density) #Changing the number density here! 4 beads per ellipsoid!
        print("L",L)
        self.L = L
        self.box = mb.Box(lengths=np.array([self.L] * 3))
        self.bead_constituents_types = ["X", "A", "T", "T"]
        super(EllipsoidChainRand, self).__init__(
            lengths=lengths, num_mols=num_mols, name=name
        )

    def _build(self, length):
        bead = mb.Compound(name="ellipsoid")
        center = mb.Compound(pos=(0, 0, 0), name="X", mass=self.bead_mass / 4)
        head = mb.Compound(
            pos=(0, 0, self.lpar + (self.bond_L / 2)),
            name="A",
            mass=self.bead_mass / 4,
        )
        tether_head = mb.Compound(
            pos=(0, 0, self.lpar), name="T", mass=self.bead_mass / 4
        )
        tether_tail = mb.Compound(
            pos=(0, 0, -self.lpar), name="T", mass=self.bead_mass / 4
        )
        bead.add([center, head, tether_head, tether_tail])
        bead.add_bond([center, head])

        chain = mb.Compound()
        last_bead = None
        rand_range = ((self.L/2) - self.lpar) #reducing step size for random walk
        print('range',rand_range)
        for i in range(length):
            translate_by = np.random.uniform(low=-1, high=1, size=(3,))
            translate_by /= np.linalg.norm(translate_by)*self.bond_L
            this_bead = mb.clone(bead)
            if last_bead:
                chain.add_bond([this_bead.children[0], last_bead.children[1]])
                chain.add_bond([this_bead.children[3], last_bead.children[2]])
                this_bead.translate(by=self.pbc(translate_by+last_bead.pos,box=([self.L]*3)))
                print('next bead',this_bead.pos)
            else:
                translate_by = np.random.uniform(low=-rand_range, high=rand_range, size=(3,))
                this_bead.translate(by=translate_by)
                print('first bead', this_bead.pos)
            chain.add(this_bead)
            last_bead = this_bead
        chain.name = f"{self.name}_{length}mer"
        #chain.box = self.box
        return chain

    def pbc(self,d,box):
        for i in range(3):
            while d[i] > box[i]/2 or d[i] < -(box[i]/2):
                print('using pbcs')
                if d[i] < -box[i]/2:
                    d[i] += box[i]
                if d[i] > box[i]/2:
                    d[i] -= box[i]
        return d

class DPD_FF(BaseHOOMDForcefield):
    def __init__(
        self,
        epsilon,
        lpar,
        lperp,
        A,
        gamma,
        kT,
        r_cut,
        angle_k=None,
        angle_theta0=None,
        bond_k=100,
        bond_r0=1.0,
    ):
        self.epsilon = epsilon
        self.lpar=lpar
        self.lperp=lperp
        self.gamma = gamma
        self.A = A
        self.kT = kT
        self.r_cut = r_cut
        self.angle_k = angle_k
        self.angle_theta0 = angle_theta0
        self.bond_k = bond_k
        self.bond_r0 = bond_r0
        hoomd_forces = self._create_forcefield()
        super(DPD_FF, self).__init__(hoomd_forces)

    def _create_forcefield(self):
        forces = []
        # Bonds
        bond = hoomd.md.bond.Harmonic()
        bond.params["T-T"] = dict(k=self.bond_k, r0=self.bond_r0)
        bond.params["A-X"] = dict(k=0, r0=0)
        forces.append(bond)
        # Angles
        if all([self.angle_k, self.angle_theta0]):
            angle = hoomd.md.angle.Harmonic()
            angle.params["X-A-X"] = dict(k=self.angle_k, t0=self.angle_theta0)
            angle.params["A-X-A"] = dict(k=0, t0=0)
            forces.append(angle)
        # DPD Pairs
        nlist = hoomd.md.nlist.Cell(buffer=0.40, exclusions=["body"])
        dpd = hoomd.md.pair.DPD(nlist=nlist,kT=self.kT,default_r_cut=self.r_cut)
        dpd.params[("X", "X")] = dict(A=self.A, gamma=self.gamma)
        # Add zero pairs
        for pair in [
            ("R", "R"),
            ("T", "T"),
            ("T", "R"),
            ("A", "A"),
            ("A", "X"),
            ("A", "T"),
            ("A", "R"),
            ("X", "R"),
            ("X", "T"),
        ]:
            dpd.params[pair] = dict(A=0,gamma=0.1)
            dpd.params[pair].r_cut = 0.0
        forces.append(dpd)
        return forces

class RandomSystem(System):
    def __init__(self, molecules, base_units=dict()):
        self.L = molecules.L
        super(RandomSystem, self).__init__(
            molecules=molecules, base_units=base_units
        )
    def _build_system(self):
        chain = self.all_molecules
        print(chain[0].pos)
        comp = mb.Compound()
        print("compund positions",comp.pos)
        comp.add(chain) #theory that this is resetting positions to (0,0,0)
        print("positions after adding particles",comp.pos)
        comp.box = mb.Box(lengths=np.array([self.L] * 3))
        return comp

In [19]:
ellipsoid_chain = EllipsoidChainRand(lengths=4,num_mols=4,lpar=0.5,bead_mass=1.0,density=0.1)
ff = DPD_FF(epsilon=1.0,lpar=0.5,lperp=0.5,A=5000,gamma=800,kT=2.0,r_cut=1.94,bond_k=2000,bond_r0=0.1)
ff.hoomd_forces
system = RandomSystem(molecules=ellipsoid_chain)
system.to_gsd('test.gsd')
gsd_path=('ellipsoid-test.gsd')
rigid_frame, rigid = create_rigid_ellipsoid_chain(
    system.hoomd_snapshot,lpar=0.5,lperp=0.5
)
ellipsoid_sim = Simulation(
    initial_state=rigid_frame,
    forcefield=ff.hoomd_forces,
    constraint=rigid,
    dt=0.0003,
    gsd_write_freq=int(100),
    gsd_file_name='trajectory.gsd',
    log_write_freq=int(100),
    log_file_name='log.txt')

ellipsoid_sim.save_restart_gsd()
ellipsoid_sim.run_NVT(n_steps=1e4, kT=2.0, tau_kt=10*ellipsoid_sim.dt)
ellipsoid_sim.flush_writers()

*Warning*: pos 0:-3.55230927467 -1.10038685799 -2.33516669273
*Warning*: fractional pos :-0.154341001373 0.29730699585 0.0698585922714
*Warning*: lo: -2.71441745758 -2.71441745758 -2.71441745758
*Warning*: hi: 2.71441745758 2.71441745758 2.71441745758
*Warning*: Not all particles were found inside the given box


L 5.4288352331898135
range 2.2144176165949068
first bead [-0.83789176  1.61403065  0.51675088]
using pbcs
using pbcs
next bead [ 1.36859928 -1.30682055  0.32370933]
using pbcs
using pbcs
using pbcs
next bead [ 1.89583058 -2.16790108 -0.144128  ]
using pbcs
using pbcs
using pbcs
next bead [-2.69506273 -2.00537349  2.35562711]
range 2.2144176165949068
first bead [ 0.992392   -1.8427294   0.59403176]
using pbcs
next bead [ 0.09131081  2.523403   -1.07031782]
using pbcs
using pbcs
next bead [-0.258766   -1.08414574  1.46086605]
using pbcs
using pbcs
using pbcs
next bead [-2.67506078 -1.78886325  0.18657536]
range 2.2144176165949068
first bead [-0.08491892 -0.13413361  2.26779838]
using pbcs
using pbcs
using pbcs
using pbcs
next bead [-0.8808171   0.86139074 -2.00268617]
using pbcs
using pbcs
using pbcs
next bead [ 2.16382635 -2.17674439 -2.53733743]
using pbcs
using pbcs
using pbcs
next bead [ 1.14461922 -1.14538874  0.89297015]
range 2.2144176165949068
first bead [2.17774717 0.78761007 1.

RuntimeError: Error initializing ParticleData

In [ ]:
snap = ellipsoid_sim.state

In [ ]:
traj = gsd.hoomd.open('trajectory.gsd','r')

In [ ]:
last_frame = traj[-1]

In [ ]:
orientations = last_frame.particles.orientation

In [ ]:
N = 16
print(last_frame.particles.typeid, last_frame.particles.types)
print("orientations of rigid bodies\n",orientations[0:N])

In [ ]:
#positions of T particles
positions = last_frame.particles.position
typids = last_frame.particles.typeid
t_idx = np.where(typids == 2)

In [ ]:
t_positions = positions[t_idx[0]]

In [ ]:
print('ellipsoid anchor positions',t_positions)